# Training and Testing A Visual Transformer Model
Here we test a visual Transfrom ViT from [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

In [1]:
import torch
import torchvision as tv
import torchvision.transforms.v2 as v2
from datasets import Country_images
from Country_dict import country_dict

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

Using cuda device


## Load our Dataset and Create our dataloaders

In [2]:
batch_size = 64
weights = tv.models.ViT_B_16_Weights.DEFAULT
transform = v2.Compose([weights.transforms(), ])

num_classes = len(country_dict)
dataset_path = "data\\compressed_dataset\\"
dataset = Country_images("data\\compressed_dataset\\country.csv",dataset_path,transform=transform)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,lengths=[0.7,0.1,0.2])
data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    'num_workers': 10,  # Number of subprocesses to use for data loading
    'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
}
train_dataloader      = torch.utils.data.DataLoader(train_dataset, **data_loader_params, shuffle=True)
val_dataloader        = torch.utils.data.DataLoader(val_dataset, **data_loader_params, shuffle=True)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, **data_loader_params, shuffle=False,in_order=True)

## Load our model

In [3]:
#hyperparameters:
lr = 0.1
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 1.0
weight_decay = 0.00001
EPOCHS = 100
#end hyperparameters

#model and optimizers
model = tv.models.vit_b_16(weights=weights, dropout=0.5)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.8)
#scales the gradients, neccessary for mixed precision data types to properly converge
scaler = torch.amp.GradScaler(device=device)
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS*len(train_dataloader))

#added data to our model for ease of use (and to prevent passing so many variables to our training function)
model.device = device
model.name = "ViT_b_16"
model.path = "Trained_Models\\ViT\\" #where to save our best model
print(model.path)
#redfine our output layer to output our classes
model.heads.head = torch.nn.Linear(in_features=model.heads.head.in_features,out_features=num_classes,bias=True)
print(model)

Trained_Models\ViT\
VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.5, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.5, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.5, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=T

## Train our Model

In [4]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel
model_train = True

if model_train:
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, scaler)
if not model_train:
    checkpoint = torch.load(model.path+model.name+"-best")
    model.load_state_dict(checkpoint)
    model = model.to(device)

EPOCH 1:
  batch 100 loss: 0.05004376545548439
  batch 200 loss: 0.033041514456272125
  batch 300 loss: 0.0266557727009058
  batch 400 loss: 0.023180032148957253
  batch 500 loss: 0.020939107984304428
LOSS train 0.020939107984304428 valid 0.007787656970322132
Accuracy: 0.2476
EPOCH 2:
  batch 100 loss: 0.011309764347970486
  batch 200 loss: 0.011052842251956463
  batch 300 loss: 0.010859115980565548
  batch 400 loss: 0.010692327283322811
  batch 500 loss: 0.01054232008755207
LOSS train 0.01054232008755207 valid 0.0075785331428050995
Accuracy: 0.2476
EPOCH 3:
  batch 100 loss: 0.009690883569419384
  batch 200 loss: 0.00952620804309845
  batch 300 loss: 0.009417369961738586
  batch 400 loss: 0.009327630512416363
  batch 500 loss: 0.00923741515725851
LOSS train 0.00923741515725851 valid 0.0074894665740430355
Accuracy: 0.2476
EPOCH 4:
  batch 100 loss: 0.008795890025794506
  batch 200 loss: 0.008688902482390404
  batch 300 loss: 0.008623488247394562
  batch 400 loss: 0.008559324778616428
 

KeyboardInterrupt: 

## Test our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_dataloader, loss_fn)

RuntimeError: Caught RuntimeError in DataLoader worker process 9.
Original Traceback (most recent call last):
  File "c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 211, in collate
    return [
           ^
  Fil